In [771]:
import os
import sys
from collections import defaultdict, Counter
import timeit
import time

import importlib
import abxportinf
import abxportinf._grouper
import abxportinf._optimize
import abxportinf._vectorizer
import abxportinf.busdef
import abxportinf.main 
import abxportinf.util
from abxportinf.busdef import BusDef

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [735]:
block_root = '/ddr'
comp_json_path = os.path.join(block_root, 'ddr.json5')
ports = abxportinf.get_ports_from_json5(comp_json_path)

Warning ('regHADDR', 'PAR_REG_ADDR_WIDTH + 2') not correctly parsed
Warning ('phy_reg_addr', '-(PAR_REG_ADDR_WIDTH)') not correctly parsed


In [736]:
bus_spec_rootdir = '/bus-defs/specs'
                                
print('loading bus defs from specs')
bus_defs = []                
for root, dirs, fnames in os.walk(bus_spec_rootdir):
    #print((root, dirs, files))  
    for fname in fnames:             
        spec_path = os.path.join(root, fname)
        if BusDef.is_spec_bus_def(spec_path):     
            print('  - loading ', spec_path)
            bus_defs.extend(
                BusDef.bus_defs_from_spec(spec_path)
            )                
print('  - done')                                            
print('loaded {} bus definitions from specs'.format(len(bus_defs)))
print('  - total req ports', sum([bd.num_req_ports for bd in bus_defs]))
print('  - total opt ports', sum([bd.num_opt_ports for bd in bus_defs]))

loading bus defs from specs
  - loading  /bus-defs/specs/amba.com/AMBA2/AHB/r3p0_1/AHB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Target/r0p0_0/AHB5Target_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Initiator/r0p0_0/AHB5Initiator_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4Stream/r0p0_1/AXI4Stream_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_RO_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE-Lite/r0p0_0/ACE-Lite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/APB4/r0p0_0/APB4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ATB/r0p0_0/ATB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACP/r0p0_0/ACP_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE/r0p0_0/ACE_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/AHBLite/r2p0_0/AHBLite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/APB/r2p0_0/APB_rtl.json5


In [789]:
def get_bus_defs():
    return list(bus_defs)
    #return list(filter(
    #    lambda bd: bd.abstract_type.name == 'AXI4_rtl',
    #    bus_defs,
    #))

In [796]:
port_group = abxportinf.get_test_port_group(ports)

stime = time.time()

bus_defs_s = get_bus_defs()
fcosts = sorted([
    (
        abxportinf._optimize.get_mapping_fcost(port_group, bus_def), 
        # (0,0,0),
        abxportinf._optimize.map_ports_to_bus(port_group, bus_def), 
        bus_def,
    )    
    for bus_def in bus_defs_s
], key=lambda x:x[0])
for fcost, (cost, mapping, cost_func), bus_def in fcosts:
    print('fcost:{}, cost:{}, driver:{}, abstract:{}'.format(
        fcost, cost, bus_def.driver_type, bus_def.abstract_type))
    
    #print('num_req:{}, num_opt:{}'.format(bus_def.num_req_ports, bus_def.num_opt_ports))
    #for (is_opt, cost), pp, bp in sorted(
    #    [((bp in set(bus_def.opt_ports), cost_func(pp, bp)), pp, bp) for pp, bp in mapping.items()],
    #    key=lambda x: x[0],
    #):
    #    print('  - cost:{}, {:15s}:{:15s} {}'.format(
    #        cost_func(pp, bp),
    #        str(pp), str(bp),
    #        'opt' if is_opt else 'req',
    #    ))
    #umap_ports = set(port_group) - set(mapping.keys())
    #umap_busports = set(bus_def.req_ports) - set(mapping.values())
    #print('  umap phy ports')
    #for port in sorted(umap_ports):
    #    print('    - ', port)
    #print('  umap bus ports')
    #for port in sorted(umap_busports):
    #    print('    - ', port)
etime = time.time()
print('time elapsed: {}s'.format(etime-stime))

fcost:19(n:0;w:14;d:5), cost:182(n:152;w:22;d:8), driver:slave, abstract:{'vendor': 'amba.com', 'library': 'AMBA2', 'name': 'AHB_rtl', 'version': 'r3p0_1'}
fcost:20(n:0;w:14;d:6), cost:286(n:257;w:21;d:8), driver:slave, abstract:{'vendor': 'amba.com', 'library': 'AMBA5', 'name': 'AHB5Target_rtl', 'version': 'r0p0_0'}
fcost:20(n:0;w:13;d:7), cost:137(n:97;w:24;d:16), driver:slave, abstract:{'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'AXI4_rtl', 'version': 'r0p0_0'}
fcost:23(n:0;w:15;d:8), cost:349(n:323;w:19;d:7), driver:slave, abstract:{'vendor': 'amba.com', 'library': 'AMBA5', 'name': 'AHB5Initiator_rtl', 'version': 'r0p0_0'}
fcost:24(n:0;w:15;d:9), cost:150(n:117;w:19;d:14), driver:slave, abstract:{'vendor': 'amba.com', 'library': 'AMBA3', 'name': 'AHBLite_rtl', 'version': 'r2p0_0'}
fcost:24(n:0;w:15;d:9), cost:223(n:190;w:19;d:14), driver:slave, abstract:{'vendor': 'amba.com', 'library': 'AMBA3', 'name': 'AHBLiteTarget_rtl', 'version': 'r2p0_0'}
fcost:25(n:0;w:16;d:9), cost:1

In [740]:
fcost, (cost, mapping, cost_func), bus_def = fcosts[0]
from abxportinf._optimize import MatchCost

def COST(mapping, ports, busdef):
    umap_ports = set(ports) - set(mapping.keys())
    umap_busports = set(busdef.req_ports) - set(mapping.values())
    cost = MatchCost.zero()          
    cost += sum([cost_func(p1, p2) for p1, p2 in mapping.items()])
    nil_port = ('', None, None)          
    cost += sum([cost_func(nil_port, p) for p in umap_ports])
    cost += sum([cost_func(nil_port, p) for p in umap_busports])
    return cost
    
print(COST(mapping, port_group, bus_def))
print(cost)
#for p1, p2 in mapping.items():
#    continue
#    #print(cost_func(p1,p2))
#costs = [cost_func(p1, p2) for p1, p2 in mapping.items()]
#umap_ports = set(bus_def.req_ports) - set(mapping.values())
#nil_port = ('', None, None)
#print(len(umap_ports))
#print(sum([cost_func(nil_port, p) for p in umap_ports]))

266(n:228;w:22;d:16)
266(n:228;w:22;d:16)


In [523]:
print((len(bus_mappings)))
for i, ((cost, mapping), bus_def)  in enumerate(sorted(bus_mappings, key=lambda x:x[0][0])):
#for i, (cost, bus_def, mapping) in enumerate(sorted(bus_mappings)):
    #if i > 5:
    #    print('BREAKING')
    #    break
    #print(bus_def.req_ports)
    #print(mapping)
    print('bus      type:', bus_def.bus_type)
    print('abstract type:', bus_def.abstract_type)
    print('driver   type:', bus_def.driver_type)
    print('num req ports:', bus_def.num_req_ports)
    print('num opt ports:', bus_def.num_opt_ports)
    for is_opt, pp, bp in sorted(
        [(bp in set(bus_def.opt_ports), pp, bp) for pp, bp in mapping.items()]
    ):
        print('  - {:15s}:{:15s} {}'.format(str(pp), str(bp), 'opt' if is_opt else 'req'))
    

34
bus      type: {'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'AXI4', 'version': 'r0p0_0'}
abstract type: {'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'AXI4_RO_rtl', 'version': 'r0p0_0'}
driver   type: slave
num req ports: 10
num opt ports: 13
  - ('axi0_AWADDR', 37, 1):('ARADDR', None, 1) req
  - ('axi0_AWBURST', 2, 1):('ARBURST', 2, 1) req
  - ('axi0_AWLEN', 8, 1):('ARLEN', 8, 1) req
  - ('axi0_AWQOS', 1, 1):('ARREADY', 1, -1) req
  - ('axi0_AWSIZE', 3, 1):('ARSIZE', 3, 1) req
  - ('axi0_AWVALID', 1, 1):('ARVALID', 1, 1) req
  - ('axi0_BREADY', 1, 1):('RREADY', 1, 1) req
  - ('axi0_WDATA', 256, 1):('RDATA', None, -1) req
  - ('axi0_WLAST', 1, 1):('RLAST', 1, -1) req
  - ('axi0_WVALID', 1, 1):('RVALID', 1, -1) req
  - ('axi0_ACLK', 1, 1):('ACLK', 1, 1)  opt
  - ('axi0_ARESETn', 1, 1):('ARESETn', 1, 1) opt
  - ('axi0_ARQOS', 1, 1):('ARQOS', 4, 1) opt
  - ('axi0_AWALLSTRB', 1, 1):('ARUSER', None, 1) opt
  - ('axi0_AWAPCMD', 1, 1):('ARCACHE', 4, 1) opt
  - ('axi0_AWCOBUF', 1